### Imports

In [2]:
import warnings
import pandas as pd
import numpy as np
import ast
import re
from datetime import date
from scipy.stats import linregress
from google.colab import drive
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind, ttest_rel

drive.mount('/content/drive')
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

Mounted at /content/drive


### Preprocessing

In [ ]:
subjective = pd.read_csv('/content/drive/My Drive/coreway_ml/Thesis - Mika/Data/raw_subjective_data_2025-11-06.csv')

subjective.drop(['hasOtherHealthProblems',
                 'hrvMeasurement',
                 'stressType',
                 'otherSymptoms_x',
                 'otherHealthProblems',
                 'mentalStressLevel',
                 'medication',
                 'otherMedication_x',
                 'otherSymptoms_y',
                 'alcoholPortions',
                 'notes',
                 'diseaseRelapsesEvaluation',
                 'generalCondition',
                 'complications',
                 'stoolsPerDay',
                 'stoolsPerNight',
                 'urgencyOfDefecation',
                 'bloodInStool',
                 'stomachPain',
                 'unformedStoolsPerDay',
                 'abdominalResistance',
                 'terraUserId',
                 'additionalIllnesses',
                 'otherKnownCatalysts',
                 'flaresPerYear',
                 'hrvMeasurementMethod',
                 'backendSymptoms',
                 'hrvMeasurementMethodName',
                 'hasAskedToConnectWearable',
                 'hasConnectedWearable',
                 'knownCatalysts',
                 'backendKnownCatalysts',
                 'connectedWearableName',
                 'hasStoma',
                 'otherAdditionalIllnesses'
                 ], axis=1, inplace=True)

# Helper Functions
def year_to_age(year_of_birth):
    current_year = date.today().year
    try:
        year = float(year_of_birth)
        if year <= 0 or year > current_year:
            return None
        return int(current_year - year)
    except (ValueError, TypeError):
        return None

# Mappings
activity_mapping = {
        'zero': 0,
        'below30min': 1,
        'below1h': 2,
        'below2h': 3,
        'below4h': 4,
        'below8h': 5,
        'above8h': 6
    }

diagnosis_mapping = {
        "colitisUlcerosa": "UC",
        "crohnsDisease": "CD",
        "Crohn's disease": "CD",
    }

gender_mapping = {
        "female": "F",
        "male": "M"
    }

alcohol_mapping = {
        "Yes": 2,
        "A little": 1,
        "No": 0,
        "Unsure": np.nan
    }

period_mapping = {
        "Yes": 1,
        "Unsure": 0,
        "False": np.nan,
        "No": 0,
    }

subjective['date'] = pd.to_datetime(subjective['date'], format='mixed').dt.normalize()
subjective['age'] = subjective['yearOfBirth'].apply(year_to_age)
subjective['hasConsumedAlcoholInLast24Hours'] = subjective['hasConsumedAlcoholInLast24Hours'].map(alcohol_mapping)
subjective['activity_dur'] = subjective['physicalEffort'].map(activity_mapping)
subjective['diagnosis'] = subjective['diagnosis'].map(diagnosis_mapping)
subjective['isOnPeriod'] = subjective['isOnPeriod'].map(period_mapping)
subjective['gender'] = subjective['gender'].map(gender_mapping)

subjective.dropna(subset=["gender", "diagnosis"], inplace=True)

# Renaming
subjective = subjective.rename(columns={
                 'userId': 'user_id',
                 'sleepQualityDegree': 'sleep',
                 'stressLevelDegree': 'stress',
                 'physicalActivityExertionDegree': 'activity_deg',
                 'symptomDegree': 'symptom_deg',
                 'hasConsumedAlcoholInLast24Hours': 'alcohol_last_24h',
                 'isOnPeriod': 'on_period',
                 'rateAsFlare': 'rate_as_flare',
                 })

# Reordering columns
subjective = subjective[[
                 'user_id',
                 'date',
                 'gender',
                 'age',
                 'diagnosis',
                 'symptoms',
                 'alcohol_last_24h',
                 'on_period',
                 'sleep',
                 'stress',
                 'activity_dur',
                 'activity_deg',
                 'symptom_deg',
                 'rate_as_flare'
                 ]]

subjective.head()

In [ ]:
def fit_full_cosinor(hrv_values, sampling_interval_minutes=5.0):

    y = np.asarray(hrv_values, dtype=float)
    y = y[~np.isnan(y)]

    # Need enough points to fit 3 parameters
    if y.size < 4:
        return np.nan, np.nan, np.nan, np.nan

    # time vector in hours
    dt = sampling_interval_minutes / 60.0
    t = np.arange(y.size, dtype=float) * dt

    # Total duration T (hours) = "night length" for this recording
    T = y.size * dt
    omega_fixed = 2.0 * np.pi / T

    # Model with fixed period T (omega fixed)
    def cosinor_model_fixed(t, mesor, amplitude, acrophase):
        return mesor + amplitude * np.cos(omega_fixed * t + acrophase)

    # Initial guesses
    mesor0 = y.mean()
    amplitude0 = (y.max() - y.min()) / 2.0
    acrophase0 = 0.0
    p0 = (mesor0, amplitude0, acrophase0)

    try:
        params, _ = curve_fit(cosinor_model_fixed, t, y, p0=p0, maxfev=5000)
        mesor, amplitude, acrophase = params
    except Exception:
        return np.nan, np.nan, np.nan, np.nan

    # Enforce amplitude >= 0 by flipping sign if necessary
    if amplitude < 0:
        amplitude = -amplitude
        acrophase = (acrophase + np.pi) % (2.0 * np.pi)

    # Peak time (max of the cosine) in hours, constrained to [0, T)
    # peak when omega*t + acrophase = 0 mod 2π -> t = -acrophase/omega
    peak_time = (-acrophase / omega_fixed) % T

    return mesor, acrophase, amplitude, peak_time

# -------------------------------------------------------------------
# Load data
# -------------------------------------------------------------------
raw_summary_wearable_data = pd.read_csv(
    '/content/drive/My Drive/coreway_ml/Thesis - Mika/Data/raw_summary_wearable_data_2025-09-26.csv'
)
raw_flattened_wearable_data = pd.read_csv(
    '/content/drive/My Drive/coreway_ml/Thesis - Mika/Data/raw_flattened_wearable_data_2025-09-26.csv'
)

# -------------------------------------------------------------------
# Transform stringified HRV list to actual list
# -------------------------------------------------------------------
raw_flattened_wearable_data['hrv_rmssd'] = raw_flattened_wearable_data['hrv_rmssd'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

# -------------------------------------------------------------------
# Standardize date columns for merging
# -------------------------------------------------------------------
raw_summary_wearable_data['date'] = pd.to_datetime(
    raw_summary_wearable_data['date'].str.slice(0, 19), errors='coerce'
)
raw_flattened_wearable_data['date'] = pd.to_datetime(
    raw_flattened_wearable_data['date'].str.slice(0, 19), errors='coerce'
)

objective = pd.merge(
    raw_summary_wearable_data,
    raw_flattened_wearable_data,
    on=['userId', 'date'],
    how='inner'
)

# -------------------------------------------------------------------
# Use end_time's calendar date as sleep date, extract HH:MM from start/end
# -------------------------------------------------------------------
objective['date'] = pd.to_datetime(objective['end_time'].str.slice(0, 10), errors='coerce')

objective[['start_time', 'end_time']] = (
    objective[['start_time', 'end_time']]
    .astype('string')
    .apply(lambda col: col.str.extract(r'T(\d{2}:\d{2})', expand=False))
)

# -------------------------------------------------------------------
# Drop unneeded columns
# -------------------------------------------------------------------
objective.drop([
    'terra_user_id',
    'provider_y',
    'sleep_score',
    'delta_temperature',
    'user_max_hr_bpm',
    'on_demand_reading',
    'breaths_start_time',
    'breaths_end_time',
    'max_breaths_per_min',
    'min_breaths_per_min',
    'duration_in_bed_seconds',
    'num_REM_events',
    'duration_long_interruption_seconds',
    'duration_short_interruption_seconds',
    'num_out_of_bed_events',
    'num_wakeup_events',
    'sleep_latency_seconds',
    'wake_up_latency_seconds',
    'max_hr_bpm',
    'min_hr_bpm',
    'bpm_array_length',
    'timestamp_intervals_seconds_hrv_rmssd',
    'hrv_rmssd_array_length',
    'level',
    'timestamp_intervals_seconds_level',
    'level_array_length',
    'percentage_array_length',
    'breaths_per_min_array_length',
    'timestamp_intervals_seconds_hrv_sdnn',
    'timestamp_intervals_seconds_bpm',
    'timestamp_intervals_seconds_percentage',
    'timestamp_intervals_seconds_breaths_per_min',
    'hrv_sdnn_array_length'
], axis=1, inplace=True)

# -------------------------------------------------------------------
# Renaming columns
# -------------------------------------------------------------------
objective = objective.rename(columns={
    'userId': 'user_id',
    'provider_x': 'provider',
    'start_time': 'start',
    'end_time': 'end',
    'avg_hr_bpm': 'avg_bpm',
    'resting_hr_bpm': 'rhr',
    'sleep_efficiency': 'sleep_eff',
    'breaths_per_min': 'breaths',
    'avg_breaths_per_min': 'avg_breaths',
    'avg_saturation_percentage': 'avg_SpO2',
    'duration_REM_sleep_state_seconds': 'dur_REM',
    'duration_asleep_state_seconds': 'dur_asleep',
    'duration_deep_sleep_state_seconds': 'dur_deep',
    'duration_light_sleep_state_seconds': 'dur_light',
    'duration_awake_state_seconds': 'dur_awake',
    'percentage': 'SpO2'
})

# -------------------------------------------------------------------
# Infer sleep length (HH:MM) and keep numeric duration in hours
# -------------------------------------------------------------------
start_dt = pd.to_datetime(objective['start'], format='%H:%M', errors='coerce')
end_dt   = pd.to_datetime(objective['end'],   format='%H:%M', errors='coerce')

# Handle crossing midnight (fixed boolean -> timedelta)
mask = (end_dt < start_dt).astype(int)
end_dt_corrected = end_dt + pd.to_timedelta(mask, unit="D")

diff = end_dt_corrected - start_dt
length_hours = diff.dt.total_seconds() / 3600.0

objective['length'] = (
    (diff.dt.seconds // 3600).astype(str).str.zfill(2) + ":" +
    ((diff.dt.seconds % 3600) // 60).astype(str).str.zfill(2)
)
objective['length_hours'] = length_hours

# -------------------------------------------------------------------
# Remove duplicates & aggregate; keep longest sleep per day
# -------------------------------------------------------------------
objective = (
    objective
    .groupby(['user_id', 'date', 'start', 'end'], as_index=False)
    .agg(lambda x: x.dropna().iloc[0] if x.notna().any() else np.nan)
)

idx_longest = objective.groupby(['user_id', 'date'])['length_hours'].idxmax()
objective = objective.loc[idx_longest].reset_index(drop=True)

# -------------------------------------------------------------------
# Convert durations from seconds to hours & compute sleep stage percentages
# -------------------------------------------------------------------
objective[['dur_asleep', 'dur_REM', 'dur_deep', 'dur_light', 'dur_awake']] /= 3600.0

objective[['REM_pct', 'deep_pct', 'light_pct']] = (
    objective[['dur_REM', 'dur_deep', 'dur_light']]
    .div(objective['dur_asleep'], axis=0) * 100.0
)

objective['sleep_eff'] = objective['sleep_eff'] * 100.0

# -------------------------------------------------------------------
# Drop rows with implausible sleep metrics
# -------------------------------------------------------------------
objective = objective[objective['dur_asleep'] >= 2]

objective = objective[
    objective['REM_pct'].between(0, 40) &
    objective['light_pct'].between(0, 90) &
    objective['deep_pct'].between(0, 40)
]

total_pct = objective['REM_pct'] + objective['light_pct'] + objective['deep_pct']
objective = objective[total_pct.between(90, 110)]

objective['sleep_eff'] = objective['sleep_eff'].where(
    objective['sleep_eff'].between(25, 100), np.nan
)

# -------------------------------------------------------------------
# Replace non-physiological values with NaNs
# -------------------------------------------------------------------
objective['avg_hrv_sdnn'] = objective['avg_hrv_sdnn'].where(
    objective['avg_hrv_sdnn'].between(5, 300), np.nan
)
objective['avg_hrv_rmssd'] = objective['avg_hrv_rmssd'].where(
    objective['avg_hrv_rmssd'].between(5, 300), np.nan
)
objective['avg_SpO2'] = objective['avg_SpO2'].where(
    objective['avg_SpO2'].between(85, 100), np.nan
)
objective['avg_bpm'] = objective['avg_bpm'].where(
    objective['avg_bpm'].between(30, 150), np.nan
)
objective['rhr'] = objective['rhr'].where(
    objective['rhr'].between(30, 150), np.nan
)

# -------------------------------------------------------------------
# Derive HRV features + cosinor features from hrv_rmssd (single pass)
# -------------------------------------------------------------------
std_list = []
cv_list = []
min_list = []
max_list = []
slope_list = []
mesor_list = []
acrophase_list = []
amplitude_list = []
peak_time_list = []

for x in objective['hrv_rmssd']:
    if isinstance(x, (list, np.ndarray, pd.Series)) and len(x) > 0:
        arr = np.asarray(x, dtype='float64')

        # Basic HRV stats
        std_val = arr.std()
        mean_val = arr.mean()
        min_val = arr.min()
        max_val = arr.max()

        if arr.size > 1:
            slope = linregress(np.arange(arr.size), arr).slope
        else:
            slope = np.nan

        # Cosinor with period exactly matching this recording's length
        mesor, acrophase, amplitude, peak_time = fit_full_cosinor(
            arr, sampling_interval_minutes=5.0
        )

        std_list.append(std_val)
        cv_list.append(std_val / mean_val if mean_val != 0 else np.nan)
        min_list.append(min_val)
        max_list.append(max_val)
        slope_list.append(slope)
        mesor_list.append(mesor)
        acrophase_list.append(acrophase)
        amplitude_list.append(amplitude)
        peak_time_list.append(peak_time)
    else:
        std_list.append(np.nan)
        cv_list.append(np.nan)
        min_list.append(np.nan)
        max_list.append(np.nan)
        slope_list.append(np.nan)
        mesor_list.append(np.nan)
        acrophase_list.append(np.nan)
        amplitude_list.append(np.nan)
        peak_time_list.append(np.nan)

objective['std_rmssd'] = std_list
objective['cv_rmssd'] = cv_list
objective['min_rmssd'] = min_list
objective['max_rmssd'] = max_list
objective['range_rmssd'] = objective['max_rmssd'] - objective['min_rmssd']
objective['slope_rmssd'] = slope_list

objective['mesor_rmssd'] = mesor_list
objective['acrophase_rmssd'] = acrophase_list
objective['amplitude_rmssd'] = amplitude_list
objective['peak_time_rmssd'] = peak_time_list

# -------------------------------------------------------------------
# Reordering columns (now including cosinor features)
# -------------------------------------------------------------------
objective = objective[[
    'user_id',
    'date',
    'provider',
    'start',
    'end',
    'length',
    'length_hours',
    'sleep_eff',
    'dur_asleep',
    'dur_REM',
    'REM_pct',
    'dur_deep',
    'deep_pct',
    'dur_light',
    'light_pct',
    'dur_awake',
    'hrv_rmssd',
    'avg_hrv_rmssd',
    'std_rmssd',
    'cv_rmssd',
    'min_rmssd',
    'max_rmssd',
    'range_rmssd',
    'slope_rmssd',
    'mesor_rmssd',
    'acrophase_rmssd',
    'amplitude_rmssd',
    'peak_time_rmssd',
    'avg_hrv_sdnn',
    'avg_bpm',
    'rhr',
    'avg_SpO2',
    'avg_breaths'
]]

# -------------------------------------------------------------------
# Sorting
# -------------------------------------------------------------------
objective = objective.sort_values(
    by=['user_id', 'date', 'start', 'end']
).reset_index(drop=True)

objective.head()

In [ ]:
merged = pd.merge(subjective, objective, on=['user_id', 'date'], how='outer', indicator=True)

merged = merged.dropna(subset=["sleep", "hrv_rmssd", "rate_as_flare"]).reset_index(drop=True)

# Keep only rows within quantile bounds
merged['hrv_len'] = merged['hrv_rmssd'].apply(len)
merged = merged[(merged['hrv_len'] >= merged['hrv_len'].quantile(0.025)) & (merged['hrv_len'] <= merged['hrv_len'].quantile(0.975))].copy().reset_index(drop=True)
merged = merged[['user_id', 'date', 'gender', 'age', 'diagnosis', 'sleep', 'rate_as_flare', 'dur_asleep', 'sleep_eff', 'REM_pct', 'deep_pct', 'light_pct', 'hrv_rmssd', 'avg_hrv_rmssd', 'std_rmssd', 'cv_rmssd', 'min_rmssd', 'max_rmssd', 'range_rmssd', 'slope_rmssd', 'mesor_rmssd', 'acrophase_rmssd', 'amplitude_rmssd', 'peak_time_rmssd']]

merged.head()

### Plots

In [ ]:
# Color codes
flare_color   = '#d62728'
unsure_color  = '#ff7f0e'
rem_color     = '#1f77b4'

plt.figure(figsize=(10, 5))

# Extract list lengths per flare category
yes_lengths = merged.loc[merged["rate_as_flare"] == "Yes", "hrv_rmssd"].dropna().apply(len)
unsure_lengths = merged.loc[merged["rate_as_flare"] == "Unsure", "hrv_rmssd"].dropna().apply(len)
no_lengths = merged.loc[merged["rate_as_flare"] == "No", "hrv_rmssd"].dropna().apply(len)

# Histogram binning
bins = 50

# Stacked histogram in the required order
plt.hist(
    [yes_lengths, unsure_lengths, no_lengths],
    bins=bins,
    stacked=True,
    color=[flare_color, unsure_color, rem_color],
    edgecolor="black",
    label=["Flare", "Unsure", "Remission"]
)

plt.xlabel("Number of HRV entries in list", fontsize=12)
plt.ylabel("Count of rows", fontsize=12)
plt.title("Distribution of HRV list lengths (RMSSD)", fontsize=14)

plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
def plot_hrv_cosinor(row):

    # --- Raw HRV data ---
    hrv = np.asarray(row["hrv_rmssd"], dtype=float)
    n_samples = len(hrv)

    dt_hours = 5.0 / 60.0
    t_hours = np.arange(n_samples) * dt_hours

    # Cosinor period (night length)
    T = n_samples * dt_hours
    omega = 2 * np.pi / T

    # ------------------------------------------------------------------
    # TRUE FITTED PARAMETERS
    # ------------------------------------------------------------------
    mesor = row["mesor_rmssd"]
    amplitude = row["amplitude_rmssd"]
    acrophase_rad = row["acrophase_rmssd"]
    peak_time_hours = row["peak_time_rmssd"]

    # ------------------------------------------------------------------
    # Cosinor model used during fitting
    # ------------------------------------------------------------------
    t_fit = np.linspace(t_hours.min(), t_hours.max(), 500)
    y_fit = mesor + amplitude * np.cos(omega * t_fit + acrophase_rad)

    # Peak HRV
    y_peak = mesor + amplitude

    # Grey color for amplitude
    amp_color = "#6E6E6E"
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.grid(which="major", linestyle="-", linewidth=1.4, alpha=0.35)

    # ===========================================================
    # Raw HRV: line + scatter
    # ===========================================================
    ax.plot(
        t_hours, hrv,
        linestyle="-", linewidth=1.5, alpha=0.85,
        label="Raw HRV (RMSSD)"
    )
    ax.scatter(t_hours, hrv, s=30, alpha=0.9)

    # Cosinor fit
    ax.plot(
        t_fit, y_fit,
        color="orange", linewidth=2.2,
        label="Cosinor fit"
    )

    # MESOR — green dashed
    ax.axhline(
        mesor,
        color="green", linestyle="--", alpha=0.7,
        label=f"MESOR = {mesor:.1f} ms"
    )

    # Peak value — red dashed
    ax.axhline(
        y_peak,
        color="red", linestyle="--", alpha=0.8
    )

    # Peak Time (Acrophase)
    ax.axvline(
        peak_time_hours,
        color="purple", linestyle=":", linewidth=2, alpha=0.9,
        label=f"Peak Time (Acrophase) = {peak_time_hours:.2f} h ({acrophase_rad:.2f} rad)"
    )

    # -----------------------------------------------------------
    # Amplitude line (dashed grey, vertical, from mesor → peak)
    # -----------------------------------------------------------
    arrow_x = 1.75
    ax.vlines(
        x=arrow_x,
        ymin=mesor,
        ymax=y_peak,
        color=amp_color,
        linestyle="--",
        linewidth=2
    )

    # Amplitude in legend
    ax.plot(
        [], [], color=amp_color, linestyle="--", linewidth=2,
        label=f"Amplitude = {amplitude:.1f} ms"
    )

    # Axis labels
    ax.set_xlabel("Time (hours since start of night)")
    ax.set_ylabel("HRV (RMSSD)")

    ax.legend(loc="upper left")
    plt.tight_layout()
    plt.show()

plot_hrv_cosinor(merged.iloc[101])

In [ ]:
def plot_hrv_stats(row):

    # ---------------------------------------------------------
    # Raw HRV samples
    # ---------------------------------------------------------
    hrv = np.asarray(row["hrv_rmssd"], dtype=float)
    n_samples = len(hrv)

    dt_minutes = 5.0
    dt_hours = dt_minutes / 60.0
    t_hours = np.arange(n_samples) * dt_hours

    # ---------------------------------------------------------
    # Stats
    # ---------------------------------------------------------
    avg = float(row["avg_hrv_rmssd"])
    std = float(row["std_rmssd"])
    cv = float(row["cv_rmssd"])
    hrv_min = float(row["min_rmssd"])
    hrv_max = float(row["max_rmssd"])
    rng = float(row["range_rmssd"])
    slope_per_5 = float(row["slope_rmssd"])

    # Trend line
    x_idx = np.arange(n_samples)
    intercept = avg - slope_per_5 * np.mean(x_idx)
    trend = intercept + slope_per_5 * x_idx

    # X-location for range indicator
    x_range = 2

    # ---------------------------------------------------------
    # Build figure
    # ---------------------------------------------------------
    fig, ax = plt.subplots(figsize=(12, 6))

    # Grid
    ax.grid(which="major", linestyle="-", linewidth=1.2, alpha=0.35)

    # Raw HRV
    ax.plot(
        t_hours, hrv,
        linestyle="-", marker="o",
        linewidth=1.5, alpha=0.9,
        label="Raw HRV (RMSSD)"
    )

    # Avg RMSSD
    ax.axhline(
        avg, color="green", linestyle="--", linewidth=1.5,
        label=f"Mean: {avg:.2f} ms"
    )

    # Min RMSSD (blue)
    ax.axhline(
        hrv_min, color="blue", linestyle="--", linewidth=1.5,
        label=f"Min: {hrv_min:.2f} ms"
    )

    # Max RMSSD (red)
    ax.axhline(
        hrv_max, color="red", linestyle="--", linewidth=1.5,
        label=f"Max: {hrv_max:.2f} ms"
    )

    # STD band
    ax.axhspan(
        avg - std, avg + std,
        facecolor="green", alpha=0.15,
        label=f"STD: ±{std:.2f} ms"
    )

    # ---------------------------------------------------------
    # Vertical RANGE RMSSD — grey dashed line
    # ---------------------------------------------------------
    ax.vlines(
        x=x_range,
        ymin=hrv_min,
        ymax=hrv_max,
        color="#6E6E6E",
        linestyle="--",
        linewidth=2
    )

    # Legend entry
    ax.plot(
        [], [], color="#6E6E6E", linestyle="--",
        label=f"Range: {rng:.2f} ms"
    )

    # Trend line
    ax.plot(
        t_hours, trend,
        color="orange", linewidth=2.0,
        label=f"Slope: {slope_per_5:.4f} ms per 5 min"
    )

    # Axis labels (no title)
    ax.set_xlabel("Time (hours since start of night)")
    ax.set_ylabel("HRV (RMSSD)")

    ax.legend(loc="upper left")
    plt.tight_layout()
    plt.show()

# Example call:
plot_hrv_stats(merged.iloc[101])

### Cosinor Analysis

1.   Without "Unsure"
2.   With "Unsure"



In [ ]:
# --------------------------------------------------
# 1) Define flare / remission and keep valid rows
# --------------------------------------------------
plot_df = merged.copy()
plot_df = plot_df[plot_df['rate_as_flare'].isin(['Yes', 'No'])].copy()
plot_df['state'] = plot_df['rate_as_flare'].map({'Yes': 'flare', 'No': 'remission'})

# Drop rows without cosinor features
plot_df = plot_df.dropna(subset=[
    'mesor_rmssd', 'amplitude_rmssd', 'acrophase_rmssd',
    'peak_time_rmssd', 'dur_asleep'
])

# --------------------------------------------------
# 1b) Summary table (NO unique ID printout now)
# --------------------------------------------------
summary_table = (
    plot_df.groupby('state')
    .agg(
        n_datapoints   = ('state', 'size'),
        n_unique_users = ('user_id', 'nunique')
    )
)

print("\n===== Summary of data used in analysis =====\n")
print(summary_table)

# --------------------------------------------------
# Split into state-specific dataframes
# --------------------------------------------------
flare_df = plot_df[plot_df['state'] == 'flare']
rem_df   = plot_df[plot_df['state'] == 'remission']

# --------------------------------------------------
# 2) Helper: mean + 95% CI
# --------------------------------------------------
def mean_ci(x):
    x = pd.Series(x).dropna()
    n = x.size
    if n == 0:
        return np.nan, np.nan, np.nan
    mean = x.mean()
    if n == 1:
        return mean, np.nan, np.nan
    se = x.std(ddof=1) / np.sqrt(n)
    return mean, mean - 1.96 * se, mean + 1.96 * se

# --------------------------------------------------
# 3) Build group-level nocturnal cosinor curves
#    (time = hours since sleep onset)
# --------------------------------------------------
T_ref = plot_df['dur_asleep'].mean()
t_grid = np.linspace(0, T_ref, 200)

def group_curve(df, t_grid, T_ref):
    mesor_mean, _, _ = mean_ci(df['mesor_rmssd'])
    amp_mean,   _, _ = mean_ci(df['amplitude_rmssd'])
    peak_mean,  _, _ = mean_ci(df['peak_time_rmssd'])
    y = mesor_mean + amp_mean * np.cos(2 * np.pi * (t_grid - peak_mean) / T_ref)
    return y, mesor_mean, amp_mean, peak_mean

y_flare, mesor_flare, amp_flare, peak_flare = group_curve(flare_df, t_grid, T_ref)
y_rem,   mesor_rem,   amp_rem,   peak_rem   = group_curve(rem_df,   t_grid, T_ref)

# --------------------------------------------------
# 4) Summary stats
# --------------------------------------------------
features = {
    'mesor_rmssd':     'MESOR',
    'amplitude_rmssd': 'Amplitude',
    'acrophase_rmssd': 'Acrophase (rad)',
    'peak_time_rmssd': 'PeakTime (h from sleep onset)'
}

group_stats = {
    state: {col: mean_ci(df_state[col]) for col in features.keys()}
    for state, df_state in plot_df.groupby('state')
}

# ---------- p-values ----------
def p_to_stars(p):
    if np.isnan(p): return "n.s."
    if p < 0.001:  return "***"
    if p < 0.01:   return "**"
    if p < 0.05:   return "*"
    return "n.s."

pvals = {}
for col in features.keys():
    x = flare_df[col].dropna()
    y = rem_df[col].dropna()
    if len(x) > 1 and len(y) > 1:
        _, p = ttest_ind(x, y, equal_var=False)
    else:
        p = np.nan
    pvals[col] = p

# --------------------------------------------------
# 5) Plot
# --------------------------------------------------
flare_color = '#d62728'
rem_color   = '#1f77b4'

fig = plt.figure(figsize=(11, 5))
gs = fig.add_gridspec(2, 3, width_ratios=[2.3, 1, 1])

# ---- Left: nocturnal HRV curves ----
ax_main = fig.add_subplot(gs[:, 0])
ax_main.plot(t_grid, y_flare, color=flare_color, label='Flare', lw=2)
ax_main.plot(t_grid, y_rem,   color=rem_color,   label='Remission', lw=2)

ax_main.set_xlabel('Hours from sleep onset')
ax_main.set_ylabel('HRV (RMSSD)')
ax_main.set_title('Nocturnal HRV cosinor fit')
ax_main.legend(frameon=False)
ax_main.grid(True, alpha=0.3)

# ---- Right: feature panels ----
axes = [
    fig.add_subplot(gs[0, 1]),
    fig.add_subplot(gs[0, 2]),
    fig.add_subplot(gs[1, 1]),
    fig.add_subplot(gs[1, 2])
]

for ax, (col, title) in zip(axes, features.items()):
    x_pos = [0, 1]
    means  = [group_stats[s][col][0] for s in ['flare', 'remission']]
    lowers = [group_stats[s][col][1] for s in ['flare', 'remission']]
    uppers = [group_stats[s][col][2] for s in ['flare', 'remission']]

    # Plot points + CIs
    for i, (m, low, high, color) in enumerate(
        zip(means, lowers, uppers, [flare_color, rem_color])
    ):
        if np.isnan(m):
            continue
        if np.isnan(low) or np.isnan(high):
            ax.plot(x_pos[i], m, 'o', color=color)
        else:
            ax.errorbar(
                x_pos[i], m,
                yerr=[[m - low], [high - m]],
                fmt='o', color=color, capsize=4
            )

    # Significance bracket
    p = pvals[col]
    stars = p_to_stars(p)

    valid_highs = [u for u in uppers if not np.isnan(u)]
    valid_lows  = [l for l in lowers if not np.isnan(l)]

    y_max = max(valid_highs) if valid_highs else max(means)
    y_min = min(valid_lows)  if valid_lows  else min(means)

    # Compute range and vertical layout
    y_range = max(y_max - y_min, 1.0)
    h = 0.08 * y_range
    y_bracket = y_max + 0.15 * y_range
    y_top = y_bracket + h * 2.5

    # Set ylim before drawing bracket/text to avoid overlap with title
    ax.set_ylim(y_min - 0.1 * y_range, y_top)

    if stars != "n.s.":
        # Bracket
        ax.plot(
            [x_pos[0], x_pos[0], x_pos[1], x_pos[1]],
            [y_bracket, y_bracket + h, y_bracket + h, y_bracket],
            lw=1, color='k'
        )
        # Stars
        ax.text(
            0.5, y_bracket + h * 0.4,
            stars,
            ha='center', va='bottom',
            fontsize=11
        )

    ax.set_xlim(-0.5, 1.5)
    ax.set_xticks(x_pos)
    ax.set_xticklabels(['Flare', 'Remission'])
    ax.set_title(title)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# --------------------------------------------------
# 1) Define flare / remission / unsure and keep valid rows
# --------------------------------------------------
plot_df = merged.copy()
plot_df = plot_df[plot_df['rate_as_flare'].isin(['Yes', 'No', 'Unsure'])].copy()
plot_df['state'] = plot_df['rate_as_flare'].map({
    'Yes':    'flare',
    'No':     'remission',
    'Unsure': 'unsure'
})

# Drop rows without cosinor features
plot_df = plot_df.dropna(subset=[
    'mesor_rmssd', 'amplitude_rmssd', 'acrophase_rmssd',
    'peak_time_rmssd', 'dur_asleep'
])

# --------------------------------------------------
# 1b) Summary table
# --------------------------------------------------
summary_table = (
    plot_df.groupby('state')
    .agg(
        n_datapoints   = ('state', 'size'),
        n_unique_users = ('user_id', 'nunique')
    )
)

print("\n===== Summary of data used in analysis =====\n")
print(summary_table)

# --------------------------------------------------
# Split into state-specific dataframes
# --------------------------------------------------
flare_df  = plot_df[plot_df['state'] == 'flare']
rem_df    = plot_df[plot_df['state'] == 'remission']
unsure_df = plot_df[plot_df['state'] == 'unsure']

# --------------------------------------------------
# 2) Helper: mean + 95% CI
# --------------------------------------------------
def mean_ci(x):
    x = pd.Series(x).dropna()
    n = x.size
    if n == 0:
        return np.nan, np.nan, np.nan
    mean = x.mean()
    if n == 1:
        return mean, np.nan, np.nan
    se = x.std(ddof=1) / np.sqrt(n)
    return mean, mean - 1.96 * se, mean + 1.96 * se

# --------------------------------------------------
# 3) Build group-level nocturnal cosinor curves
#    (time = hours since sleep onset)
# --------------------------------------------------
T_ref = plot_df['dur_asleep'].mean()
t_grid = np.linspace(0, T_ref, 200)

def group_curve(df, t_grid, T_ref):
    mesor_mean, _, _ = mean_ci(df['mesor_rmssd'])
    amp_mean,   _, _ = mean_ci(df['amplitude_rmssd'])
    peak_mean,  _, _ = mean_ci(df['peak_time_rmssd'])
    y = mesor_mean + amp_mean * np.cos(2 * np.pi * (t_grid - peak_mean) / T_ref)
    return y, mesor_mean, amp_mean, peak_mean

y_flare,  *_ = group_curve(flare_df,  t_grid, T_ref)
y_rem,    *_ = group_curve(rem_df,    t_grid, T_ref)
y_unsure, *_ = group_curve(unsure_df, t_grid, T_ref)

# --------------------------------------------------
# 4) Summary stats
# --------------------------------------------------
features = {
    'mesor_rmssd':     'MESOR',
    'amplitude_rmssd': 'Amplitude',
    'acrophase_rmssd': 'Acrophase (rad)',
    'peak_time_rmssd': 'PeakTime (h from sleep onset)'
}

group_stats = {
    state: {col: mean_ci(df_state[col]) for col in features.keys()}
    for state, df_state in plot_df.groupby('state')
}

# ---------- p-values (Flare vs Remission only) ----------
def p_to_stars(p):
    if np.isnan(p): return "n.s."
    if p < 0.001:  return "***"
    if p < 0.01:   return "**"
    if p < 0.05:   return "*"
    return "n.s."

pvals = {}
for col in features.keys():
    x = flare_df[col].dropna()
    y = rem_df[col].dropna()
    if len(x) > 1 and len(y) > 1:
        _, p = ttest_ind(x, y, equal_var=False)
    else:
        p = np.nan
    pvals[col] = p

# --------------------------------------------------
# 5) Plot
# --------------------------------------------------
flare_color   = '#d62728'
rem_color     = '#1f77b4'
unsure_color  = '#ff7f0e'

# NEW ORDER
state_order = ['remission', 'unsure', 'flare']

state_colors = {
    'flare':     flare_color,
    'remission': rem_color,
    'unsure':    unsure_color
}

state_labels = {
    'flare':     'Flare',
    'remission': 'Remission',
    'unsure':    'Unsure'
}

fig = plt.figure(figsize=(11, 5))
gs = fig.add_gridspec(2, 3, width_ratios=[2.3, 1, 1])

# ---- Left: nocturnal HRV curves ----
ax_main = fig.add_subplot(gs[:, 0])
ax_main.plot(t_grid, y_rem,    color=rem_color,    label='Remission', lw=2)
ax_main.plot(t_grid, y_unsure, color=unsure_color, label='Unsure',    lw=2)
ax_main.plot(t_grid, y_flare,  color=flare_color,  label='Flare',     lw=2)

ax_main.set_xlabel('Hours from sleep onset')
ax_main.set_ylabel('HRV (RMSSD)')
ax_main.set_title('Nocturnal HRV cosinor fit')
ax_main.legend(frameon=False)
ax_main.grid(True, alpha=0.3)

# ---- Right: feature panels ----
axes = [
    fig.add_subplot(gs[0, 1]),
    fig.add_subplot(gs[0, 2]),
    fig.add_subplot(gs[1, 1]),
    fig.add_subplot(gs[1, 2])
]

for ax, (col, title) in zip(axes, features.items()):

    states = state_order
    x_pos = np.arange(len(states))

    means  = [group_stats[s][col][0] for s in states]
    lowers = [group_stats[s][col][1] for s in states]
    uppers = [group_stats[s][col][2] for s in states]

    # Plot means and CIs
    for i, s in enumerate(states):
        m, low, high = means[i], lowers[i], uppers[i]
        color = state_colors[s]
        if np.isnan(m):
            continue
        if np.isnan(low) or np.isnan(high):
            ax.plot(x_pos[i], m, 'o', color=color)
        else:
            ax.errorbar(
                x_pos[i], m,
                yerr=[[m - low], [high - m]],
                fmt='o', color=color, capsize=4
            )

    p = pvals[col]
    stars = p_to_stars(p)

    valid_highs = [u for u in uppers if not np.isnan(u)]
    valid_lows  = [l for l in lowers if not np.isnan(l)]
    valid_means = [m for m in means  if not np.isnan(m)]

    y_max = max(valid_highs) if valid_highs else max(valid_means, default=1)
    y_min = min(valid_lows)  if valid_lows  else min(valid_means, default=0)

    y_range = max(y_max - y_min, 1.0)
    h = 0.08 * y_range
    y_bracket = y_max + 0.15 * y_range
    y_top = y_bracket + h * 2.5

    ax.set_ylim(y_min - 0.1 * y_range, y_top)

    if stars != "n.s.":
        ax.plot(
            [x_pos[0], x_pos[0], x_pos[2], x_pos[2]],
            [y_bracket, y_bracket + h, y_bracket + h, y_bracket],
            lw=1, color='k'
        )
        ax.text(
            (x_pos[0] + x_pos[2]) / 2.0,
            y_bracket + h * 0.4,
            stars,
            ha='center', va='bottom',
            fontsize=11
        )

    ax.set_xlim(-0.5, len(states) - 0.5)
    ax.set_xticks(x_pos)
    ax.set_xticklabels([state_labels[s] for s in states])
    ax.set_title(title)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Sub-Analysis restricted to User's that contribute both Flare/Remission Data

1.   Welch Approach
2.   Pairwise Approach



In [ ]:
# Identify users who have both "Yes" and "No"
valid_users = (
    merged.groupby('user_id')['rate_as_flare']
    .apply(lambda x: {'Yes', 'No'}.issubset(set(x)))
)

# Filter the dataframe to keep only those users
merged = merged[merged['user_id'].isin(valid_users[valid_users].index)]

# --------------------------------------------------
# 1) Define flare / remission and keep valid rows
# --------------------------------------------------
plot_df = merged.copy()
plot_df = plot_df[plot_df['rate_as_flare'].isin(['Yes', 'No'])].copy()
plot_df['state'] = plot_df['rate_as_flare'].map({'Yes': 'flare', 'No': 'remission'})

# Drop rows without cosinor features
plot_df = plot_df.dropna(subset=[
    'mesor_rmssd', 'amplitude_rmssd', 'acrophase_rmssd', 'peak_time_rmssd',
    'dur_asleep'
])

# --------------------------------------------------
# 1b) Summary table + unique user ID printout
# --------------------------------------------------
summary_table = (
    plot_df.groupby('state')
    .agg(
        n_datapoints   = ('state', 'size'),
        n_unique_users = ('user_id', 'nunique')
    )
)

print("\n===== Summary of data used in analysis =====\n")
print(summary_table)

print("\n--- Unique user IDs contributing to plot ---\n")
unique_users = sorted(plot_df['user_id'].unique())
for u in unique_users:
    print(u)

# --------------------------------------------------
# Split into state-specific dataframes
# --------------------------------------------------
flare_df = plot_df[plot_df['state'] == 'flare']
rem_df   = plot_df[plot_df['state'] == 'remission']

# --------------------------------------------------
# 2) Helper: mean + 95% CI
# --------------------------------------------------
def mean_ci(x):
    x = pd.Series(x).dropna()
    n = x.size
    if n == 0:
        return np.nan, np.nan, np.nan
    mean = x.mean()
    if n == 1:
        return mean, np.nan, np.nan
    se = x.std(ddof=1) / np.sqrt(n)
    return mean, mean - 1.96 * se, mean + 1.96 * se

# --------------------------------------------------
# 3) Build group-level nocturnal cosinor curves
#    (time = hours since sleep onset)
# --------------------------------------------------
T_ref = plot_df['dur_asleep'].mean()
t_grid = np.linspace(0, T_ref, 200)

def group_curve(df, t_grid, T_ref):
    mesor_mean, _, _      = mean_ci(df['mesor_rmssd'])
    amp_mean,   _, _      = mean_ci(df['amplitude_rmssd'])
    peak_mean,  _, _      = mean_ci(df['peak_time_rmssd'])

    # Cosinor: peak at peak_mean
    y = mesor_mean + amp_mean * np.cos(2 * np.pi * (t_grid - peak_mean) / T_ref)
    return y, mesor_mean, amp_mean, peak_mean

y_flare,  mesor_flare, amp_flare, peak_flare = group_curve(flare_df, t_grid, T_ref)
y_rem,    mesor_rem,   amp_rem,   peak_rem   = group_curve(rem_df,   t_grid, T_ref)

# --------------------------------------------------
# 4) Summary stats for MESOR / Amplitude / Acrophase / PeakTime
# --------------------------------------------------
features = {
    'mesor_rmssd':     'MESOR',
    'amplitude_rmssd': 'Amplitude',
    'acrophase_rmssd': 'Acrophase (rad)',
    'peak_time_rmssd': 'PeakTime (h from sleep onset)'
}

group_stats = {}
for state, df_state in plot_df.groupby('state'):
    group_stats[state] = {}
    for col in features.keys():
        group_stats[state][col] = mean_ci(df_state[col])

# ---------- compute p-values and star mapping ----------
def p_to_stars(p):
    if np.isnan(p):
        return "n.s."
    if p < 0.001:
        return "***"
    elif p < 0.01:
        return "**"
    elif p < 0.05:
        return "*"
    else:
        return "n.s."

pvals = {}
for col in features.keys():
    x = flare_df[col].dropna()
    y = rem_df[col].dropna()
    if len(x) > 1 and len(y) > 1:
        stat, p = ttest_ind(x, y, equal_var=False)
    else:
        p = np.nan
    pvals[col] = p

# --------------------------------------------------
# 5) Plot
# --------------------------------------------------
flare_color = '#d62728'
rem_color   = '#1f77b4'

fig = plt.figure(figsize=(11, 5))
gs = fig.add_gridspec(2, 3, width_ratios=[2.3, 1, 1])

# ---- Left: nocturnal HRV curves ----
ax_main = fig.add_subplot(gs[:, 0])
ax_main.plot(t_grid, y_flare,  color=flare_color, label='Flare',      lw=2)
ax_main.plot(t_grid, y_rem,    color=rem_color,   label='Remission',  lw=2)

ax_main.set_xlabel('Hours from sleep onset')
ax_main.set_ylabel('HRV (RMSSD)')
ax_main.set_title('Nocturnal HRV cosinor fit')
ax_main.legend(frameon=False)
ax_main.grid(True, alpha=0.3)

# ---- Right: 2×2 panels with mean ± 95% CI ----
axes = [
    fig.add_subplot(gs[0, 1]),
    fig.add_subplot(gs[0, 2]),
    fig.add_subplot(gs[1, 1]),
    fig.add_subplot(gs[1, 2])
]

for ax, (col, title) in zip(axes, features.items()):
    x_pos = [0, 1]
    means, lowers, uppers = [], [], []

    for state in ['flare', 'remission']:
        mean, low, high = group_stats[state][col]
        means.append(mean); lowers.append(low); uppers.append(high)

    # Plot points and error bars
    for i, (m, low, high, color) in enumerate(
        zip(means, lowers, uppers, [flare_color, rem_color])
    ):
        if np.isnan(m):
            continue
        if np.isnan(low) or np.isnan(high):
            ax.plot(x_pos[i], m, 'o', color=color)
        else:
            ax.errorbar(
                x_pos[i], m,
                yerr=[[m - low], [high - m]],
                fmt='o', color=color, capsize=4
            )

    # --- significance bracket + stars, if any ---
    p = pvals[col]
    stars = p_to_stars(p)

    # y-range for placing the annotation nicely
    valid_highs = [u for u in uppers if not np.isnan(u)]
    valid_lows  = [l for l in lowers if not np.isnan(l)]
    if valid_highs:
        y_max = max(valid_highs)
    else:  # fall back to means if no CI
        y_max = max([m for m in means if not np.isnan(m)])

    if valid_lows:
        y_min = min(valid_lows)
    else:
        y_min = min([m for m in means if not np.isnan(m)])

    y_range = max(y_max - y_min, 1.0)
    h = 0.08 * y_range
    y_bracket = y_max + 0.15 * y_range
    y_top = y_bracket + h * 2.5

    # Set ylim before drawing bracket/text so they fit comfortably
    ax.set_ylim(y_min - 0.1 * y_range, y_top)

    if stars != "n.s.":
        # draw bracket
        ax.plot(
            [x_pos[0], x_pos[0], x_pos[1], x_pos[1]],
            [y_bracket, y_bracket + h, y_bracket + h, y_bracket],
            lw=1, color='k'
        )
        # add star text above bracket, centered between groups
        ax.text(
            0.5, y_bracket + h * 0.4,
            stars,
            ha='center', va='bottom',
            fontsize=11
        )

    # --- x-axis formatting ---
    ax.set_xlim(-0.5, 1.5)
    ax.set_xticks(x_pos)
    ax.set_xticklabels(['Flare', 'Remission'])
    ax.set_title(title)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# --------------------------------------------------
# 0) Filter users that have both "Yes" and "No" in the original merged df
# --------------------------------------------------
valid_users = (
    merged.groupby('user_id')['rate_as_flare']
    .apply(lambda x: {'Yes', 'No'}.issubset(set(x)))
)

# Filter the dataframe to keep only those users
merged = merged[merged['user_id'].isin(valid_users[valid_users].index)]

# --------------------------------------------------
# 1) Define flare / remission and keep valid rows
# --------------------------------------------------
plot_df = merged.copy()
plot_df = plot_df[plot_df['rate_as_flare'].isin(['Yes', 'No'])].copy()
plot_df['state'] = plot_df['rate_as_flare'].map({'Yes': 'flare', 'No': 'remission'})

# Drop rows without cosinor features
plot_df = plot_df.dropna(subset=[
    'mesor_rmssd', 'amplitude_rmssd', 'acrophase_rmssd', 'peak_time_rmssd',
    'dur_asleep'
])

# --------------------------------------------------
# 1b) Enforce pairwise design AFTER dropping NaNs:
#     keep only users who still have BOTH flare and remission
# --------------------------------------------------
valid_pair_users = (
    plot_df.groupby('user_id')['state']
    .apply(lambda s: {'flare', 'remission'}.issubset(set(s)))
)

plot_df = plot_df[plot_df['user_id'].isin(valid_pair_users[valid_pair_users].index)]

# --------------------------------------------------
# 1c) Summary table + unique user ID printout
#     (still based on the row-level data, but only for paired users)
# --------------------------------------------------
summary_table = (
    plot_df.groupby('state')
    .agg(
        n_datapoints   = ('state', 'size'),
        n_unique_users = ('user_id', 'nunique')
    )
)

print("\n===== Summary of data used in analysis =====\n")
print(summary_table)

print("\n--- Unique user IDs contributing to plot ---\n")
unique_users = sorted(plot_df['user_id'].unique())
for u in unique_users:
    print(u)

# --------------------------------------------------
# 1d) Collapse to per-user, per-state means (pairwise data)
# --------------------------------------------------
user_state_means = (
    plot_df.groupby(['user_id', 'state'])
    [['mesor_rmssd', 'amplitude_rmssd', 'acrophase_rmssd', 'peak_time_rmssd', 'dur_asleep']]
    .mean()
    .reset_index()
)

# Split into state-specific dataframes (now per-user, not per-night)
flare_df = user_state_means[user_state_means['state'] == 'flare']
rem_df   = user_state_means[user_state_means['state'] == 'remission']

# --------------------------------------------------
# 2) Helper: mean + 95% CI
# --------------------------------------------------
def mean_ci(x):
    x = pd.Series(x).dropna()
    n = x.size
    if n == 0:
        return np.nan, np.nan, np.nan
    mean = x.mean()
    if n == 1:
        return mean, np.nan, np.nan
    se = x.std(ddof=1) / np.sqrt(n)
    return mean, mean - 1.96 * se, mean + 1.96 * se

# --------------------------------------------------
# 3) Build group-level nocturnal cosinor curves
#    (time = hours since sleep onset)
#    NOW based on per-user means (each user = 1 weight)
# --------------------------------------------------
T_ref = user_state_means['dur_asleep'].mean()
t_grid = np.linspace(0, T_ref, 200)

def group_curve(df, t_grid, T_ref):
    mesor_mean, _, _      = mean_ci(df['mesor_rmssd'])
    amp_mean,   _, _      = mean_ci(df['amplitude_rmssd'])
    peak_mean,  _, _      = mean_ci(df['peak_time_rmssd'])

    # Cosinor: peak at peak_mean
    y = mesor_mean + amp_mean * np.cos(2 * np.pi * (t_grid - peak_mean) / T_ref)
    return y, mesor_mean, amp_mean, peak_mean

y_flare,  mesor_flare, amp_flare, peak_flare = group_curve(flare_df, t_grid, T_ref)
y_rem,    mesor_rem,   amp_rem,   peak_rem   = group_curve(rem_df,   t_grid, T_ref)

# --------------------------------------------------
# 4) Summary stats for MESOR / Amplitude / Acrophase / PeakTime
#    based on per-user means
# --------------------------------------------------
features = {
    'mesor_rmssd':     'MESOR',
    'amplitude_rmssd': 'Amplitude',
    'acrophase_rmssd': 'Acrophase (rad)',
    'peak_time_rmssd': 'PeakTime (h from sleep onset)'
}

group_stats = {}
for state, df_state in user_state_means.groupby('state'):
    group_stats[state] = {}
    for col in features.keys():
        group_stats[state][col] = mean_ci(df_state[col])

# ---------- compute p-values and star mapping (PAIRED t-test) ----------
def p_to_stars(p):
    if np.isnan(p):
        return "n.s."
    if p < 0.001:
        return "***"
    elif p < 0.01:
        return "**"
    elif p < 0.05:
        return "*"
    else:
        return "n.s."

pvals = {}
for col in features.keys():
    # Pivot to user_id × state so we can do a paired comparison
    wide = (
        user_state_means
        .pivot(index='user_id', columns='state', values=col)
        .dropna(subset=['flare', 'remission'])
    )

    if wide.shape[0] > 1:
        x = wide['flare']
        y = wide['remission']
        stat, p = ttest_rel(x, y)
    else:
        p = np.nan

    pvals[col] = p

# --------------------------------------------------
# 5) Plot
# --------------------------------------------------
flare_color = '#d62728'
rem_color   = '#1f77b4'

fig = plt.figure(figsize=(11, 5))
gs = fig.add_gridspec(2, 3, width_ratios=[2.3, 1, 1])

# ---- Left: nocturnal HRV curves (per-user-based cosinor) ----
ax_main = fig.add_subplot(gs[:, 0])
ax_main.plot(t_grid, y_flare,  color=flare_color, label='Flare',      lw=2)
ax_main.plot(t_grid, y_rem,    color=rem_color,   label='Remission',  lw=2)

ax_main.set_xlabel('Hours from sleep onset')
ax_main.set_ylabel('HRV (RMSSD)')
ax_main.set_title('Nocturnal HRV cosinor fit (per-user means)')
ax_main.legend(frameon=False)
ax_main.grid(True, alpha=0.3)

# ---- Right: 2×2 panels with mean ± 95% CI (per-user means) ----
axes = [
    fig.add_subplot(gs[0, 1]),
    fig.add_subplot(gs[0, 2]),
    fig.add_subplot(gs[1, 1]),
    fig.add_subplot(gs[1, 2])
]

for ax, (col, title) in zip(axes, features.items()):
    x_pos = [0, 1]
    means, lowers, uppers = [], [], []

    for state in ['flare', 'remission']:
        mean, low, high = group_stats[state][col]
        means.append(mean); lowers.append(low); uppers.append(high)

    # Plot points and error bars
    for i, (m, low, high, color) in enumerate(
        zip(means, lowers, uppers, [flare_color, rem_color])
    ):
        if np.isnan(m):
            continue
        if np.isnan(low) or np.isnan(high):
            ax.plot(x_pos[i], m, 'o', color=color)
        else:
            ax.errorbar(
                x_pos[i], m,
                yerr=[[m - low], [high - m]],
                fmt='o', color=color, capsize=4
            )

    # --- significance bracket + stars, if any ---
    p = pvals[col]
    stars = p_to_stars(p)

    # y-range for placing the annotation nicely
    valid_highs = [u for u in uppers if not np.isnan(u)]
    valid_lows  = [l for l in lowers if not np.isnan(l)]
    if valid_highs:
        y_max = max(valid_highs)
    else:
        y_max = max([m for m in means if not np.isnan(m)])

    if valid_lows:
        y_min = min(valid_lows)
    else:
        y_min = min([m for m in means if not np.isnan(m)])

    y_range = max(y_max - y_min, 1.0)
    h = 0.08 * y_range
    y_bracket = y_max + 0.15 * y_range
    y_top = y_bracket + h * 2.5

    # Set ylim before drawing bracket/text so they fit comfortably
    ax.set_ylim(y_min - 0.1 * y_range, y_top)

    if stars != "n.s.":
        # draw bracket
        ax.plot(
            [x_pos[0], x_pos[0], x_pos[1], x_pos[1]],
            [y_bracket, y_bracket + h, y_bracket + h, y_bracket],
            lw=1, color='k'
        )
        # add star text above bracket, centered between groups
        ax.text(
            0.5, y_bracket + h * 0.4,
            stars,
            ha='center', va='bottom',
            fontsize=11
        )

    # --- x-axis formatting ---
    ax.set_xlim(-0.5, 1.5)
    ax.set_xticks(x_pos)
    ax.set_xticklabels(['Flare', 'Remission'])
    ax.set_title(title)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Further Analysis where restricted to User's contributing "Yes", "Unsure" AND "No" in rate_as_flare

1.   Welch Approach
2.   Pairwise Approach

In [ ]:
# ---------------------------------------------
# 0) Restrict to users who have all 3 states
# ---------------------------------------------
valid_users = (
    merged.groupby('user_id')['rate_as_flare']
    .apply(lambda x: {'Yes', 'No', 'Unsure'}.issubset(set(x)))
)

merged = merged[merged['user_id'].isin(valid_users[valid_users].index)]

# --------------------------------------------------
# 1) Define flare / remission / unsure and keep valid rows
# --------------------------------------------------
plot_df = merged.copy()
plot_df = plot_df[plot_df['rate_as_flare'].isin(['Yes', 'No', 'Unsure'])].copy()

plot_df['state'] = plot_df['rate_as_flare'].map({
    'Yes': 'flare',
    'No': 'remission',
    'Unsure': 'unsure'
})

# Drop rows without cosinor features
plot_df = plot_df.dropna(subset=[
    'mesor_rmssd', 'amplitude_rmssd', 'acrophase_rmssd',
    'peak_time_rmssd', 'dur_asleep'
])

# --------------------------------------------------
# 1b) Summary table + unique user ID printout
# --------------------------------------------------
summary_table = (
    plot_df.groupby('state')
    .agg(
        n_datapoints = ('state', 'size'),
        n_unique_users = ('user_id', 'nunique')
    )
)

print("\n===== Summary of data used in analysis =====\n")
print(summary_table)

print("\n--- Unique user IDs contributing to plot ---\n")
unique_users = sorted(plot_df['user_id'].unique())
for u in unique_users:
    print(u)

# --------------------------------------------------
# Split into state-specific dataframes
# --------------------------------------------------
flare_df  = plot_df[plot_df['state'] == 'flare']
rem_df    = plot_df[plot_df['state'] == 'remission']
unsure_df = plot_df[plot_df['state'] == 'unsure']

# --------------------------------------------------
# 2) Helper: mean + 95% CI
# --------------------------------------------------
def mean_ci(x):
    x = pd.Series(x).dropna()
    n = x.size
    if n == 0:
        return np.nan, np.nan, np.nan
    mean = x.mean()
    if n == 1:
        return mean, np.nan, np.nan
    se = x.std(ddof=1) / np.sqrt(n)
    return mean, mean - 1.96 * se, mean + 1.96 * se

# --------------------------------------------------
# 3) Group-level cosinor curves
# --------------------------------------------------
T_ref = plot_df['dur_asleep'].mean()
t_grid = np.linspace(0, T_ref, 200)

def group_curve(df, t_grid, T_ref):
    mesor_mean, _, _ = mean_ci(df['mesor_rmssd'])
    amp_mean,   _, _ = mean_ci(df['amplitude_rmssd'])
    peak_mean,  _, _ = mean_ci(df['peak_time_rmssd'])
    y = mesor_mean + amp_mean * np.cos(
        2 * np.pi * (t_grid - peak_mean) / T_ref
    )
    return y, mesor_mean, amp_mean, peak_mean

y_flare,  mesor_flare,  amp_flare,  peak_flare  = group_curve(flare_df,  t_grid, T_ref)
y_rem,    mesor_rem,    amp_rem,    peak_rem    = group_curve(rem_df,    t_grid, T_ref)
y_unsure, mesor_unsure, amp_unsure, peak_unsure = group_curve(unsure_df, t_grid, T_ref)

# --------------------------------------------------
# 4) Summary stats + pairwise tests
# --------------------------------------------------
features = {
    'mesor_rmssd':     'MESOR',
    'amplitude_rmssd': 'Amplitude',
    'acrophase_rmssd': 'Acrophase (rad)',
    'peak_time_rmssd': 'PeakTime (h from sleep onset)'
}

group_stats = {
    state: {col: mean_ci(df[col]) for col in features.keys()}
    for state, df in plot_df.groupby('state')
}

def p_to_stars(p):
    if np.isnan(p): return "n.s."
    if p < 0.001: return "***"
    if p < 0.01:  return "**"
    if p < 0.05:  return "*"
    return "n.s."

states_for_plot = ['remission', 'unsure', 'flare']
pairs = [
    ('remission', 'unsure'),
    ('remission', 'flare'),
    ('unsure', 'flare')
]

pvals = {col: {} for col in features.keys()}
for col in features.keys():
    for s1, s2 in pairs:
        x = plot_df[plot_df['state'] == s1][col].dropna()
        y = plot_df[plot_df['state'] == s2][col].dropna()
        if len(x) > 1 and len(y) > 1:
            _, p = ttest_ind(x, y, equal_var=False)
        else:
            p = np.nan
        pvals[col][(s1, s2)] = p

# --------------------------------------------------
# 5) Plot
# --------------------------------------------------
flare_color   = '#d62728'
rem_color     = '#1f77b4'
unsure_color  = '#ff7f0e'

fig = plt.figure(figsize=(11, 5))
gs = fig.add_gridspec(2, 3, width_ratios=[2.3, 1, 1])

# ---- Left: HRV curves ----
ax_main = fig.add_subplot(gs[:, 0])
ax_main.plot(t_grid, y_rem,    color=rem_color,    label='Remission', lw=2)
ax_main.plot(t_grid, y_unsure, color=unsure_color, label='Unsure',    lw=2)
ax_main.plot(t_grid, y_flare,  color=flare_color,  label='Flare',     lw=2)

ax_main.set_xlabel('Hours from sleep onset')
ax_main.set_ylabel('HRV (RMSSD)')
ax_main.set_title('Nocturnal HRV cosinor fit')
ax_main.legend(frameon=False)
ax_main.grid(True, alpha=0.3)

# ---- Right: 2×2 CI panels ----
axes = [
    fig.add_subplot(gs[0, 1]),
    fig.add_subplot(gs[0, 2]),
    fig.add_subplot(gs[1, 1]),
    fig.add_subplot(gs[1, 2])
]

colors_for_plot = [rem_color, unsure_color, flare_color]
state_to_x = {s: i for i, s in enumerate(states_for_plot)}

for ax, (col, title) in zip(axes, features.items()):
    x_pos = range(3)
    means = [group_stats[s][col][0] for s in states_for_plot]
    lows  = [group_stats[s][col][1] for s in states_for_plot]
    highs = [group_stats[s][col][2] for s in states_for_plot]

    # Plot points
    for i, (m, low, high, color) in enumerate(zip(means, lows, highs, colors_for_plot)):
        if np.isnan(m):
            continue
        if np.isnan(low) or np.isnan(high):
            ax.plot(i, m, 'o', color=color)
        else:
            ax.errorbar(
                i, m,
                yerr=[[m - low], [high - m]],
                fmt='o', color=color, capsize=4
            )

    # Determine y-range
    valid_highs = [v for v in highs if not np.isnan(v)]
    valid_lows  = [v for v in lows  if not np.isnan(v)]
    y_max = max(valid_highs) if valid_highs else max(means)
    y_min = min(valid_lows)  if valid_lows  else min(means)
    y_range = max(y_max - y_min, 1.0)

    h = 0.06 * y_range
    gap = 0.16 * y_range
    start_y = y_max + 0.05 * y_range

    # Pre-set ylim so brackets + stars have room under the title
    n_pairs = len(pairs)
    highest_bracket_y = start_y + (n_pairs - 1) * gap + h * 1.2
    y_top = highest_bracket_y + 0.05 * y_range
    ax.set_ylim(y_min - 0.1 * y_range, y_top)

    # Brackets
    for i_pair, (s1, s2) in enumerate(pairs):
        p = pvals[col][(s1, s2)]
        stars = p_to_stars(p)
        if stars == "n.s.":
            continue

        x1 = state_to_x[s1]
        x2 = state_to_x[s2]
        y = start_y + i_pair * gap

        ax.plot(
            [x1, x1, x2, x2],
            [y, y + h, y + h, y],
            lw=1, color='k'
        )
        ax.text(
            (x1 + x2) / 2,
            y + h * 1.03,
            stars,
            ha='center', va='bottom',
            fontsize=9
        )

    ax.set_xlim(-0.5, 2.5)
    ax.set_xticks([0, 1, 2])
    ax.set_xticklabels(['Remission', 'Unsure', 'Flare'])
    ax.set_title(title)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# ---------------------------------------------
# 0) Restrict to users who have all 3 states
# ---------------------------------------------
valid_users = (
    merged.groupby('user_id')['rate_as_flare']
    .apply(lambda x: {'Yes', 'No', 'Unsure'}.issubset(set(x)))
)

merged = merged[merged['user_id'].isin(valid_users[valid_users].index)]

# --------------------------------------------------
# 1) Define flare / remission / unsure and keep valid rows
# --------------------------------------------------
plot_df = merged.copy()
plot_df = plot_df[plot_df['rate_as_flare'].isin(['Yes', 'No', 'Unsure'])].copy()

plot_df['state'] = plot_df['rate_as_flare'].map({
    'Yes': 'flare',
    'No': 'remission',
    'Unsure': 'unsure'
})

# Drop rows without cosinor features
plot_df = plot_df.dropna(subset=[
    'mesor_rmssd', 'amplitude_rmssd', 'acrophase_rmssd',
    'peak_time_rmssd', 'dur_asleep'
])

# --------------------------------------------------
# 1b) Enforce users still have all 3 states AFTER dropping NaNs
# --------------------------------------------------
valid_pair_users = (
    plot_df.groupby('user_id')['state']
    .apply(lambda s: {'flare', 'remission', 'unsure'}.issubset(set(s)))
)

plot_df = plot_df[plot_df['user_id'].isin(valid_pair_users[valid_pair_users].index)]

# --------------------------------------------------
# 1c) Summary table + unique user ID printout (row-level, but only paired users)
# --------------------------------------------------
summary_table = (
    plot_df.groupby('state')
    .agg(
        n_datapoints   = ('state', 'size'),
        n_unique_users = ('user_id', 'nunique')
    )
)

print("\n===== Summary of data used in analysis =====\n")
print(summary_table)

print("\n--- Unique user IDs contributing to plot ---\n")
unique_users = sorted(plot_df['user_id'].unique())
for u in unique_users:
    print(u)

# --------------------------------------------------
# 1d) Collapse to per-user, per-state means (pairwise data)
# --------------------------------------------------
user_state_means = (
    plot_df.groupby(['user_id', 'state'])
    [['mesor_rmssd', 'amplitude_rmssd', 'acrophase_rmssd',
      'peak_time_rmssd', 'dur_asleep']]
    .mean()
    .reset_index()
)

# Split into state-specific dataframes (now per-user, not per-night)
flare_df  = user_state_means[user_state_means['state'] == 'flare']
rem_df    = user_state_means[user_state_means['state'] == 'remission']
unsure_df = user_state_means[user_state_means['state'] == 'unsure']

# --------------------------------------------------
# 2) Helper: mean + 95% CI
# --------------------------------------------------
def mean_ci(x):
    x = pd.Series(x).dropna()
    n = x.size
    if n == 0:
        return np.nan, np.nan, np.nan
    mean = x.mean()
    if n == 1:
        return mean, np.nan, np.nan
    se = x.std(ddof=1) / np.sqrt(n)
    return mean, mean - 1.96 * se, mean + 1.96 * se

# --------------------------------------------------
# 3) Group-level cosinor curves (per-user means)
# --------------------------------------------------
T_ref = user_state_means['dur_asleep'].mean()
t_grid = np.linspace(0, T_ref, 200)

def group_curve(df, t_grid, T_ref):
    mesor_mean, _, _ = mean_ci(df['mesor_rmssd'])
    amp_mean,   _, _ = mean_ci(df['amplitude_rmssd'])
    peak_mean,  _, _ = mean_ci(df['peak_time_rmssd'])

    y = mesor_mean + amp_mean * np.cos(
        2 * np.pi * (t_grid - peak_mean) / T_ref
    )
    return y, mesor_mean, amp_mean, peak_mean

y_flare,  mesor_flare,  amp_flare,  peak_flare  = group_curve(flare_df,  t_grid, T_ref)
y_rem,    mesor_rem,    amp_rem,    peak_rem    = group_curve(rem_df,    t_grid, T_ref)
y_unsure, mesor_unsure, amp_unsure, peak_unsure = group_curve(unsure_df, t_grid, T_ref)

# --------------------------------------------------
# 4) Summary stats + pairwise tests (paired across users)
# --------------------------------------------------
features = {
    'mesor_rmssd':     'MESOR',
    'amplitude_rmssd': 'Amplitude',
    'acrophase_rmssd': 'Acrophase (rad)',
    'peak_time_rmssd': 'PeakTime (h from sleep onset)'
}

group_stats = {
    state: {col: mean_ci(df[col]) for col in features.keys()}
    for state, df in user_state_means.groupby('state')
}

def p_to_stars(p):
    if np.isnan(p): return "n.s."
    if p < 0.001: return "***"
    if p < 0.01:  return "**"
    if p < 0.05:  return "*"
    return "n.s."

states_for_plot = ['remission', 'unsure', 'flare']
pairs = [
    ('remission', 'unsure'),
    ('remission', 'flare'),
    ('unsure', 'flare')
]

pvals = {col: {} for col in features.keys()}
for col in features.keys():
    # Wide format: user_id × state, then paired comparisons
    wide = (
        user_state_means
        .pivot(index='user_id', columns='state', values=col)
    )

    for s1, s2 in pairs:
        sub = wide[[s1, s2]].dropna()
        if sub.shape[0] > 1:
            x = sub[s1]
            y = sub[s2]
            _, p = ttest_rel(x, y)
        else:
            p = np.nan
        pvals[col][(s1, s2)] = p

# --------------------------------------------------
# 5) Plot
# --------------------------------------------------
flare_color   = '#d62728'
rem_color     = '#1f77b4'
unsure_color  = '#ff7f0e'

fig = plt.figure(figsize=(11, 5))
gs = fig.add_gridspec(2, 3, width_ratios=[2.3, 1, 1])

# ---- Left: HRV curves ----
ax_main = fig.add_subplot(gs[:, 0])
ax_main.plot(t_grid, y_rem,    color=rem_color,    label='Remission', lw=2)
ax_main.plot(t_grid, y_unsure, color=unsure_color, label='Unsure',    lw=2)
ax_main.plot(t_grid, y_flare,  color=flare_color,  label='Flare',     lw=2)

ax_main.set_xlabel('Hours from sleep onset')
ax_main.set_ylabel('HRV (RMSSD)')
ax_main.set_title('Nocturnal HRV cosinor fit (per-user means)')
ax_main.legend(frameon=False)
ax_main.grid(True, alpha=0.3)

# ---- Right: 2×2 CI panels ----
axes = [
    fig.add_subplot(gs[0, 1]),
    fig.add_subplot(gs[0, 2]),
    fig.add_subplot(gs[1, 1]),
    fig.add_subplot(gs[1, 2])
]

colors_for_plot = [rem_color, unsure_color, flare_color]
state_to_x = {s: i for i, s in enumerate(states_for_plot)}

for ax, (col, title) in zip(axes, features.items()):
    x_pos = range(3)
    means = [group_stats[s][col][0] for s in states_for_plot]
    lows  = [group_stats[s][col][1] for s in states_for_plot]
    highs = [group_stats[s][col][2] for s in states_for_plot]

    # Plot points
    for i, (m, low, high, color) in enumerate(zip(means, lows, highs, colors_for_plot)):
        if np.isnan(m):
            continue
        if np.isnan(low) or np.isnan(high):
            ax.plot(i, m, 'o', color=color)
        else:
            ax.errorbar(
                i, m,
                yerr=[[m - low], [high - m]],
                fmt='o', color=color, capsize=4
            )

    # Determine y-range
    valid_highs = [v for v in highs if not np.isnan(v)]
    valid_lows  = [v for v in lows  if not np.isnan(v)]
    y_max = max(valid_highs) if valid_highs else max(means)
    y_min = min(valid_lows)  if valid_lows  else min(means)
    y_range = max(y_max - y_min, 1.0)

    h = 0.06 * y_range
    gap = 0.16 * y_range
    start_y = y_max + 0.05 * y_range

    # Pre-set ylim so brackets + stars have room under the title
    n_pairs = len(pairs)
    highest_bracket_y = start_y + (n_pairs - 1) * gap + h * 1.2
    y_top = highest_bracket_y + 0.05 * y_range
    ax.set_ylim(y_min - 0.1 * y_range, y_top)

    # Brackets
    for i_pair, (s1, s2) in enumerate(pairs):
        p = pvals[col][(s1, s2)]
        stars = p_to_stars(p)
        if stars == "n.s.":
            continue

        x1 = state_to_x[s1]
        x2 = state_to_x[s2]
        y = start_y + i_pair * gap

        ax.plot(
            [x1, x1, x2, x2],
            [y, y + h, y + h, y],
            lw=1, color='k'
        )
        ax.text(
            (x1 + x2) / 2,
            y + h * 1.03,
            stars,
            ha='center', va='bottom',
            fontsize=9
        )

    ax.set_xlim(-0.5, 2.5)
    ax.set_xticks([0, 1, 2])
    ax.set_xticklabels(['Remission', 'Unsure', 'Flare'])
    ax.set_title(title)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()